In [ ]:
# streaming will receive 0 records after receiving something each batch interval because of conn.close()
import socket

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(('localhost', 9999))
    s.listen(1)
    with open('stream.log', 'rb') as f:
        from os import SEEK_SET, SEEK_CUR, SEEK_END
        f.seek(0, SEEK_END)
        pos = f.tell()
        while pos > 0:
            f.seek(pos-1, SEEK_SET)
            if f.read(1) == b'\n':
                break
            pos -= 1
        count = 0
        while True:
            block = f.read()
            end = block.rfind(b'\n') + 1
            size = len(block)
            block = block[:end]
            print(f'block {count}: {len(block)} bytes')
            f.seek(end-size, SEEK_CUR)
            try:
                conn, addr = s.accept()
                conn.sendall(block)
                count += 1
            finally:
                conn.close()